# ITS Question Service – Endpoint Validation

Use this notebook to exercise every documented Flask endpoint exposed by the service.


## Usage

1. Start the ITS Question Service locally (default base URL: `http://localhost:5200`).
2. Ensure the API instance has Supabase credentials with read/write permissions; many tests insert sample data.
3. Optionally set the `ITS_QUESTION_SERVICE_BASE_URL` environment variable before running the notebook to target a remote deployment.
4. Run the cells top to bottom. The summary at the end highlights any unexpected status codes.


In [52]:
import os
import json
import uuid
from datetime import datetime
from urllib.parse import urljoin

import pandas as pd
import requests

BASE_URL = os.environ.get("ITS_QUESTION_SERVICE_BASE_URL", "http://localhost:5200")
SESSION = requests.Session()
SESSION.headers.update({
    "Accept": "application/json",
    "User-Agent": "its-question-service-notebook/1.0"
})

RESULTS = []
CONTEXT = {}

print(f"Target base URL: {BASE_URL}")


Target base URL: http://localhost:5200


In [53]:
from typing import Any, Iterable, Dict
from copy import deepcopy

try:
    from IPython.display import display
except ImportError:
    def display(obj):
        print(obj)

def build_url(path: str) -> str:
    base = BASE_URL.rstrip('/') + '/'
    relative = path.lstrip('/')
    return urljoin(base, relative)


def parse_response_body(response: requests.Response) -> Any:
    try:
        return response.json()
    except ValueError:
        return response.text


def flatten_body_preview(body: Any, limit: int = 400) -> str:
    if isinstance(body, (dict, list)):
        try:
            snippet = json.dumps(body, default=str)
        except TypeError:
            snippet = str(body)
    else:
        snippet = str(body)
    return snippet if len(snippet) <= limit else snippet[:limit] + "..."


def flatten_request_payload(kwargs: Dict[str, Any]) -> Any:
    if "json" in kwargs:
        return kwargs["json"]
    if "data" in kwargs:
        return kwargs["data"]
    return None


def ensure_set(value: Any) -> set:
    if isinstance(value, Iterable) and not isinstance(value, (str, bytes, dict, set)):
        return set(value)
    return {value}


def run_test(name: str,
             method: str,
             path: str,
             expected_status: Any = 200,
             extra_expected: Any = None,
             note: str = None,
             timeout: int = 60,
             **kwargs) -> Dict[str, Any]:
    request_kwargs = deepcopy(kwargs)
    headers = request_kwargs.pop("headers", {})
    merged_headers = {**SESSION.headers, **headers}
    url = build_url(path)

    response = SESSION.request(
        method=method.upper(),
        url=url,
        timeout=timeout,
        headers=merged_headers,
        **request_kwargs
    )
    body = parse_response_body(response)
    expected_codes = ensure_set(expected_status)
    if extra_expected is not None:
        expected_codes |= ensure_set(extra_expected)
    ok = response.status_code in expected_codes

    record = {
        "name": name,
        "method": method.upper(),
        "path": path,
        "status": response.status_code,
        "expected": sorted(expected_codes),
        "ok": ok,
        "note": note,
        "request_payload": flatten_request_payload(kwargs),
        "response_preview": flatten_body_preview(body)
    }
    RESULTS.append(record)

    status_label = "✅" if ok else "⚠️"
    print(f"{status_label} {name} -> {response.status_code}")

    return {
        "ok": ok,
        "status": response.status_code,
        "body": body,
        "headers": dict(response.headers),
        "record": record
    }


def first_item(payload: Any) -> Dict[str, Any]:
    if isinstance(payload, list) and payload:
        return payload[0]
    if isinstance(payload, dict) and "data" in payload and isinstance(payload["data"], list) and payload["data"]:
        return payload["data"][0]
    return payload if isinstance(payload, dict) else {}


def extract_id(payload: Any) -> Any:
    item = first_item(payload)
    if isinstance(item, dict):
        return item.get("id")
    return None



In [54]:
# Seed core hierarchy data used across tests
run_timestamp = datetime.utcnow().strftime("%Y%m%d%H%M%S")
unique_tag = uuid.uuid4().hex[:8]

CONTEXT["run_timestamp"] = run_timestamp
CONTEXT["unique_tag"] = unique_tag
CONTEXT["test_user_id"] = f"notebook-user-{unique_tag}"

print(f"Run timestamp: {run_timestamp}")
print(f"Unique tag: {unique_tag}")
print(f"Notebook test user id: {CONTEXT['test_user_id']}")

school_exam_resp = run_test(
    name="Create school exam",
    method="POST",
    path="/api/exams",
    expected_status=201,
    json={
        "name": f"Notebook School Exam {run_timestamp}",
        "description": "Created by endpoint coverage notebook",
        "exam_type": "school"
    }
)
if school_exam_resp["ok"]:
    school_exam = first_item(school_exam_resp["body"])
    CONTEXT["school_exam"] = school_exam
    CONTEXT["school_exam_id"] = school_exam.get("id")

competitive_exam_resp = run_test(
    name="Create competitive exam",
    method="POST",
    path="/api/exams",
    expected_status=201,
    json={
        "name": f"Notebook Competitive Exam {run_timestamp}",
        "description": "Created by endpoint coverage notebook",
        "exam_type": "competitive"
    }
)
if competitive_exam_resp["ok"]:
    competitive_exam = first_item(competitive_exam_resp["body"])
    CONTEXT["competitive_exam"] = competitive_exam
    CONTEXT["competitive_exam_id"] = competitive_exam.get("id")

if CONTEXT.get("school_exam_id"):
    class_resp = run_test(
        name="Create school class",
        method="POST",
        path="/api/classes",
        expected_status=201,
        json={
            "exam_id": CONTEXT["school_exam_id"],
            "name": f"Notebook Class {unique_tag}",
            "description": "Notebook generated class for hierarchy tests",
            "class_number": 10,
            "section": "A"
        }
    )
    if class_resp["ok"]:
        class_entry = first_item(class_resp["body"])
        CONTEXT["class"] = class_entry
        CONTEXT["class_id"] = class_entry.get("id")

if CONTEXT.get("competitive_exam_id"):
    subject_resp = run_test(
        name="Create competitive subject",
        method="POST",
        path="/api/subjects",
        expected_status=201,
        json={
            "name": f"Notebook Subject {unique_tag}",
            "description": "Subject tied to competitive exam",
            "exam_id": CONTEXT["competitive_exam_id"]
        }
    )
    if subject_resp["ok"]:
        subject_entry = first_item(subject_resp["body"])
        CONTEXT["subject"] = subject_entry
        CONTEXT["subject_id"] = subject_entry.get("id")
        CONTEXT["subject_name"] = subject_entry.get("name")

if CONTEXT.get("class_id"):
    school_subject_resp = run_test(
        name="Create school subject",
        method="POST",
        path="/api/subjects",
        expected_status=201,
        json={
            "name": f"Notebook School Subject {unique_tag}",
            "description": "Subject tied to class hierarchy",
            "class_id": CONTEXT["class_id"]
        }
    )
    if school_subject_resp["ok"]:
        school_subject_entry = first_item(school_subject_resp["body"])
        CONTEXT["school_subject"] = school_subject_entry
        CONTEXT["school_subject_id"] = school_subject_entry.get("id")

if CONTEXT.get("subject_id"):
    chapter_resp = run_test(
        name="Create chapter",
        method="POST",
        path="/api/chapters",
        expected_status=201,
        json={
            "subject_id": CONTEXT["subject_id"],
            "name": f"Notebook Chapter {unique_tag}",
            "description": "Chapter generated for testing"
        }
    )
    if chapter_resp["ok"]:
        chapter_entry = first_item(chapter_resp["body"])
        CONTEXT["chapter"] = chapter_entry
        CONTEXT["chapter_id"] = chapter_entry.get("id")

if CONTEXT.get("chapter_id"):
    topic_resp = run_test(
        name="Create topic",
        method="POST",
        path="/api/topics",
        expected_status=201,
        json={
            "chapter_id": CONTEXT["chapter_id"],
            "name": f"Notebook Topic {unique_tag}",
            "description": "Topic generated for testing"
        }
    )
    if topic_resp["ok"]:
        topic_entry = first_item(topic_resp["body"])
        CONTEXT["topic"] = topic_entry
        CONTEXT["topic_id"] = topic_entry.get("id")

if CONTEXT.get("topic_id"):
    concept_resp = run_test(
        name="Create concept",
        method="POST",
        path="/api/concepts",
        expected_status=201,
        json={
            "topic_id": CONTEXT["topic_id"],
            "name": f"Notebook Concept {unique_tag}",
            "description": "Concept generated for testing"
        }
    )
    if concept_resp["ok"]:
        concept_entry = first_item(concept_resp["body"])
        CONTEXT["concept"] = concept_entry
        CONTEXT["concept_id"] = concept_entry.get("id")

if CONTEXT.get("topic_id"):
    topic_attr_resp = run_test(
        name="Add topic attributes",
        method="POST",
        path=f"/api/topic/{CONTEXT['topic_id']}/attributes",
        expected_status=201,
        json=[{
            "name": f"Notebook Attribute {unique_tag}",
            "description": "Primary attribute for notebook questions"
        }]
    )
    if topic_attr_resp["ok"] and isinstance(topic_attr_resp["body"], dict):
        created_attributes = topic_attr_resp["body"].get("attributes") or []
        if created_attributes:
            CONTEXT["topic_attribute"] = created_attributes[0]
            CONTEXT["topic_attribute_id"] = created_attributes[0].get("id")

required_keys = ["competitive_exam_id", "subject_id", "chapter_id", "topic_id", "concept_id", "topic_attribute_id"]
missing = [key for key in required_keys if not CONTEXT.get(key)]
if missing:
    raise RuntimeError(f"Missing context entries after hierarchy setup: {missing}")
else:
    print("Core hierarchy seeded successfully.")



Run timestamp: 20251004080817
Unique tag: cada2316
Notebook test user id: notebook-user-cada2316
✅ Create school exam -> 201
✅ Create competitive exam -> 201
✅ Create school class -> 201
✅ Create competitive subject -> 201
✅ Create school subject -> 201
✅ Create chapter -> 201
✅ Create topic -> 201
✅ Create concept -> 201
✅ Add topic attributes -> 201
Core hierarchy seeded successfully.


In [55]:
# Attribute endpoints
topic_attr_list_resp = run_test(
    name="List topic attributes",
    method="GET",
    path=f"/api/topic/{CONTEXT['topic_id']}/attributes",
    expected_status=200
)
if isinstance(topic_attr_list_resp["body"], list) and topic_attr_list_resp["body"]:
    CONTEXT["topic_attributes"] = topic_attr_list_resp["body"]

run_test(
    name="List concept attributes (deprecated endpoint)",
    method="GET",
    path=f"/api/concept/{CONTEXT['concept_id']}/attributes",
    expected_status=200,
    note="Endpoint kept for backward compatibility."
)

run_test(
    name="Hierarchy attribute lookup",
    method="GET",
    path="/api/hierarchy/attributes",
    expected_status=200,
    params={"concept_id": CONTEXT["concept_id"]}
)

direct_attr_resp = run_test(
    name="Create attribute via /api/attributes",
    method="POST",
    path="/api/attributes",
    expected_status=201,
    json={
        "name": f"Notebook Direct Attribute {CONTEXT['unique_tag']}",
        "description": "Attribute created directly through /api/attributes",
        "topic_id": CONTEXT["topic_id"]
    }
)
if direct_attr_resp["ok"] and isinstance(direct_attr_resp["body"], dict):
    CONTEXT["direct_attribute"] = direct_attr_resp["body"]
    CONTEXT["direct_attribute_id"] = direct_attr_resp["body"].get("id")

if CONTEXT.get("direct_attribute_id"):
    run_test(
        name="Update direct attribute",
        method="PUT",
        path=f"/api/attributes/{CONTEXT['direct_attribute_id']}",
        expected_status=200,
        json={"description": "Updated via notebook coverage"}
    )
    run_test(
        name="Delete direct attribute",
        method="DELETE",
        path=f"/api/attributes/{CONTEXT['direct_attribute_id']}",
        expected_status=200
    )



✅ List topic attributes -> 200
✅ List concept attributes (deprecated endpoint) -> 200
✅ Hierarchy attribute lookup -> 200
✅ Create attribute via /api/attributes -> 201
✅ Update direct attribute -> 200
✅ Delete direct attribute -> 200


In [38]:
# Hierarchy traversal endpoints
if CONTEXT.get("school_exam_id"):
    run_test(
        name="List classes for exam",
        method="GET",
        path=f"/api/exams/{CONTEXT['school_exam_id']}/classes",
        expected_status=200
    )

if CONTEXT.get("class_id"):
    run_test(
        name="Get class details",
        method="GET",
        path=f"/api/classes/{CONTEXT['class_id']}",
        expected_status=200
    )
    run_test(
        name="List class subjects",
        method="GET",
        path=f"/api/classes/{CONTEXT['class_id']}/subjects",
        expected_status=200
    )

run_test(
    name="Hierarchy exams listing",
    method="GET",
    path="/api/hierarchy/exams",
    expected_status=200
)

run_test(
    name="Hierarchy subjects listing",
    method="GET",
    path="/api/hierarchy/subjects",
    expected_status=200,
    params={"exam_id": CONTEXT["competitive_exam_id"]}
)

run_test(
    name="Hierarchy chapters listing",
    method="GET",
    path="/api/hierarchy/chapters",
    expected_status=200,
    params={"subject_id": CONTEXT["subject_id"]}
)

run_test(
    name="Hierarchy topics listing",
    method="GET",
    path="/api/hierarchy/topics",
    expected_status=200,
    params={"chapter_id": CONTEXT["chapter_id"]}
)

run_test(
    name="Hierarchy concepts listing",
    method="GET",
    path="/api/hierarchy/concepts",
    expected_status=200,
    params={"topic_id": CONTEXT["topic_id"]}
)

run_test(
    name="Hierarchy tree snapshot",
    method="GET",
    path="/api/hierarchy/tree",
    expected_status=200
)

run_test(
    name="Hierarchy children for subject",
    method="GET",
    path=f"/api/hierarchy/subject/{CONTEXT['subject_id']}/children",
    expected_status=200
)

run_test(
    name="Hierarchy chain for topic",
    method="GET",
    path=f"/api/hierarchy/topic/{CONTEXT['topic_id']}/chain",
    expected_status=200
)

hier_ensure_resp = run_test(
    name="Ensure concept exists",
    method="POST",
    path="/api/hierarchy/ensure",
    expected_status=200,
    json={
        "level": "concept",
        "name": f"Ensured Concept {CONTEXT['unique_tag']}",
        "description": "Created if missing by ensure endpoint",
        "parent_id": CONTEXT["topic_id"]
    },
    note="Endpoint returns existing element if already present."
)
if hier_ensure_resp["ok"] and isinstance(hier_ensure_resp["body"], dict):
    ensured_element = hier_ensure_resp["body"].get("element")
    if isinstance(ensured_element, list) and ensured_element:
        ensured_element = ensured_element[0]
    if isinstance(ensured_element, dict):
        CONTEXT["ensured_concept_id"] = ensured_element.get("id")



✅ List classes for exam -> 200
✅ Get class details -> 200
✅ List class subjects -> 200
✅ Hierarchy exams listing -> 200
✅ Hierarchy subjects listing -> 200
✅ Hierarchy chapters listing -> 200
✅ Hierarchy topics listing -> 200
✅ Hierarchy concepts listing -> 200
✅ Hierarchy tree snapshot -> 200
✅ Hierarchy children for subject -> 200
✅ Hierarchy chain for topic -> 200
✅ Ensure concept exists -> 200


In [56]:
# Question management, search, and analytics endpoints
CONTEXT.setdefault("all_question_ids", [])

question_payload = {
    "question": {
        "content": f"What is the notebook marker {CONTEXT['unique_tag']}?",
        "options": ["Option A", "Option B", "Option C", "Option D"],
        "correct_answer": "Option A",
        "exam_id": CONTEXT["competitive_exam_id"],
        "subject_id": CONTEXT["subject_id"],
        "chapter_id": CONTEXT["chapter_id"],
        "topic_id": CONTEXT["topic_id"],
        "concept_id": CONTEXT["concept_id"],
        "difficulty": 0.45,
        "discrimination": 1.1,
        "guessing": 0.25
    },
    "selected_attributes": [
        {"attribute_id": CONTEXT["topic_attribute_id"], "value": True}
    ],
    "create_new_attributes": []
}

question_create_resp = run_test(
    name="Create question with selected attributes",
    method="POST",
    path="/api/questions/create-with-attributes",
    expected_status=201,
    json=question_payload
)
if question_create_resp["ok"] and isinstance(question_create_resp["body"], dict):
    created_question = question_create_resp["body"].get("question", {})
    CONTEXT["question"] = created_question
    CONTEXT["question_id"] = created_question.get("id")
    if CONTEXT["question_id"]:
        CONTEXT["all_question_ids"].append(CONTEXT["question_id"])
        CONTEXT["question_content"] = created_question.get("content")

batch_questions_payload = {
    "questions": [
        {
            "content": f"Batch question {i} for {CONTEXT['unique_tag']}",
            "options": ["Option A", "Option B", "Option C", "Option D"],
            "correct_answer": "Option A",
            "exam_id": CONTEXT["competitive_exam_id"],
            "subject_id": CONTEXT["subject_id"],
            "chapter_id": CONTEXT["chapter_id"],
            "topic_id": CONTEXT["topic_id"],
            "concept_id": CONTEXT["concept_id"],
            "difficulty": 0.3 + 0.1 * i,
            "discrimination": 1.0,
            "guessing": 0.25,
            "attributes": [
                {"attribute_id": CONTEXT["topic_attribute_id"], "value": True}
            ]
        }
        for i in range(1, 3)
    ]
}

batch_create_resp = run_test(
    name="Batch create questions",
    method="POST",
    path="/api/questions/batch",
    expected_status=201,
    json=batch_questions_payload
)
if batch_create_resp["ok"] and isinstance(batch_create_resp["body"], dict):
    batch_questions = batch_create_resp["body"].get("questions", [])
    for q in batch_questions:
        if isinstance(q, dict) and q.get("id"):
            CONTEXT["all_question_ids"].append(q["id"])

if CONTEXT["all_question_ids"]:
    run_test(
        name="Batch fetch questions",
        method="POST",
        path="/api/questions/batch-get",
        expected_status=200,
        json={"question_ids": CONTEXT["all_question_ids"]}
    )

run_test(
    name="Query questions by topic",
    method="GET",
    path="/api/questions",
    expected_status=200,
    params={"topic_id": CONTEXT["topic_id"]}
)

if CONTEXT.get("question_id"):
    run_test(
        name="Get question by id",
        method="GET",
        path=f"/api/questions/{CONTEXT['question_id']}",
        expected_status=200
    )

run_test(
    name="Hierarchy questions listing",
    method="GET",
    path=f"/api/hierarchy/topic/{CONTEXT['topic_id']}/questions",
    expected_status=200,
    params={"page": 1, "page_size": 10}
)

run_test(
    name="Hierarchy enhanced questions",
    method="GET",
    path=f"/api/hierarchy/topic/{CONTEXT['topic_id']}/questions/enhanced",
    expected_status=200,
    params={"page": 1, "page_size": 10}
)

run_test(
    name="Item bank slice by topic",
    method="GET",
    path=f"/api/item-bank/topic/{CONTEXT['topic_id']}",
    expected_status=200,
    params={"page": 1, "page_size": 10}
)

run_test(
    name="Hierarchy stats for topic",
    method="GET",
    path=f"/api/hierarchy/topic/{CONTEXT['topic_id']}/stats",
    expected_status=200
)

run_test(
    name="Export EduCDM for topic",
    method="GET",
    path=f"/api/export/educdm/topic/{CONTEXT['topic_id']}",
    expected_status=200
)

run_test(
    name="Search hierarchy by keyword",
    method="GET",
    path="/api/search/hierarchy",
    expected_status=200,
    params={
        "query": (CONTEXT.get("subject_name") or "Notebook").split()[0],
        "level": "subjects"
    }
)

run_test(
    name="Search questions by text",
    method="GET",
    path="/api/search/questions",
    expected_status=200,
    params={
        "topic_id": CONTEXT["topic_id"],
        "text_search": "Batch question"
    }
)

run_test(
    name="Exam question count",
    method="GET",
    path=f"/api/hierarchy/exam/{CONTEXT['competitive_exam_id']}/question-count",
    expected_status=200
)

run_test(
    name="Subject question count",
    method="GET",
    path=f"/api/hierarchy/subject/{CONTEXT['subject_id']}/question-count",
    expected_status=200
)

run_test(
    name="Chapter question count",
    method="GET",
    path=f"/api/hierarchy/chapter/{CONTEXT['chapter_id']}/question-count",
    expected_status=200
)

run_test(
    name="Topic question count",
    method="GET",
    path=f"/api/hierarchy/topic/{CONTEXT['topic_id']}/question-count",
    expected_status=200
)

run_test(
    name="Concept question count",
    method="GET",
    path=f"/api/hierarchy/concept/{CONTEXT['concept_id']}/question-count",
    expected_status=200
)



✅ Create question with selected attributes -> 201
✅ Batch create questions -> 201
✅ Batch fetch questions -> 200
✅ Query questions by topic -> 200
✅ Get question by id -> 200
✅ Hierarchy questions listing -> 200
✅ Hierarchy enhanced questions -> 200
✅ Item bank slice by topic -> 200
✅ Hierarchy stats for topic -> 200
✅ Export EduCDM for topic -> 200
✅ Search hierarchy by keyword -> 200
✅ Search questions by text -> 200
✅ Exam question count -> 200
✅ Subject question count -> 200
✅ Chapter question count -> 200
✅ Topic question count -> 200
✅ Concept question count -> 200


{'ok': True,
 'status': 200,
 'body': {'concept_id': 'd8117670-972f-476f-8b83-62e869710fc3',
  'total_question_count': 3},
 'headers': {'Server': 'gunicorn',
  'Date': 'Sat, 04 Oct 2025 08:08:36 GMT',
  'Connection': 'close',
  'Content-Type': 'application/json',
  'Content-Length': '79',
  'Access-Control-Allow-Origin': '*'},
 'record': {'name': 'Concept question count',
  'method': 'GET',
  'path': '/api/hierarchy/concept/d8117670-972f-476f-8b83-62e869710fc3/question-count',
  'status': 200,
  'expected': [200],
  'ok': True,
  'note': None,
  'request_payload': None,
  'response_preview': '{"concept_id": "d8117670-972f-476f-8b83-62e869710fc3", "total_question_count": 3}'}}

In [57]:
# ============================================================
# PYQ Upload, Search, and Session Endpoints
# ============================================================

from io import BytesIO

print("=" * 60)
print("SECTION 1: PYQ Template and Filter Options")
print("=" * 60)

run_test(
    name="Download PYQ template",
    method="GET",
    path="/api/pyq/template/download",
    expected_status=200,
    note="Returns an Excel workbook template."
)

run_test(
    name="PYQ filter options",
    method="GET",
    path="/api/pyq/filters/options",
    expected_status=200
)

# ============================================================
print("\n" + "=" * 60)
print("SECTION 2: Upload PYQ Questions")
print("=" * 60)

# Define PYQ question payload
pyq_single_payload = {
    "content": f"Notebook PYQ question {CONTEXT['unique_tag']}?",
    "options": ["Option A", "Option B", "Option C", "Option D"],
    "correct_answer": "Option A",
    "exam_id": CONTEXT["competitive_exam_id"],
    "subject_id": CONTEXT["subject_id"],
    "chapter_id": CONTEXT["chapter_id"],
    "topic_id": CONTEXT["topic_id"],
    "concept_id": CONTEXT["concept_id"],
    "metadata": {
        "year": datetime.utcnow().year,
        "exam_session": "Spring",
        "paper_code": f"NB-{CONTEXT['unique_tag']}",
        "question_number": "1",
        "marks_allocated": 2,
        "time_allocated": 3,
        "solution": "Sample explanation generated by notebook.",
        "source": "Notebook",
        "tags": ["notebook", "pyq"],
        "difficulty_level": "Medium",
        "question_type": "MCQ"
    },
    "attributes": [
        {"attribute_id": CONTEXT["topic_attribute_id"], "value": True}
    ]
}

# Upload single PYQ question
pyq_single_resp = run_test(
    name="Upload single PYQ question",
    method="POST",
    path="/api/pyq/upload/single",
    expected_status=201,
    json=pyq_single_payload
)
if isinstance(pyq_single_resp["body"], dict) and pyq_single_resp["body"].get("success"):
    pyq_question = pyq_single_resp["body"].get("question", {})
    CONTEXT["pyq_question_id"] = pyq_question.get("id")
    print(f"  → Uploaded PYQ question ID: {CONTEXT['pyq_question_id']}")

# Upload bulk PYQ questions
pyq_bulk_payload = {
    "questions": [
        {
            **pyq_single_payload,
            "content": f"Bulk PYQ question {i} for {CONTEXT['unique_tag']}",
            "metadata": {
                **pyq_single_payload["metadata"],
                "question_number": str(i + 1)
            }
        }
        for i in range(2)
    ]
}
run_test(
    name="Upload bulk PYQ questions",
    method="POST",
    path="/api/pyq/upload/bulk",
    expected_status=200,
    json=pyq_bulk_payload
)

# Upload via Excel (currently disabled - uncomment to enable)
excel_df = pd.DataFrame([
    {
        "content": f"Excel PYQ question {i} for {CONTEXT['unique_tag']}",
        "options": json.dumps(["Option A", "Option B", "Option C", "Option D"]),
        "correct_answer": "Option A",
        "year": datetime.utcnow().year,
        "exam_session": "Spring",
        "paper_code": f"XNB-{CONTEXT['unique_tag']}-{i}",
        "question_number": str(i + 10),
        "marks_allocated": 3,
        "time_allocated": 4,
        "solution": "Notebook Excel import solution.",
        "source": "Notebook Excel",
        "tags": "excel,pyq,notebook",
        "difficulty_level": "Medium",
        "question_type": "MCQ",
        "exam_id": CONTEXT["competitive_exam_id"],
        "subject_id": CONTEXT["subject_id"],
        "chapter_id": CONTEXT["chapter_id"],
        "topic_id": CONTEXT["topic_id"],
        "concept_id": CONTEXT["concept_id"]
    }
    for i in range(2)
])
excel_buffer = BytesIO()
# excel_df.to_excel(excel_buffer, index=False)  # Disabled - requires openpyxl
excel_buffer.seek(0)

try:
    run_test(
        name="Upload PYQ questions via Excel",
        method="POST",
        path="/api/pyq/upload/excel",
        expected_status=200,
        files={"file": ("pyq_import.xlsx", excel_buffer, "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")},
        data={}
    )
finally:
    excel_buffer.close()

# ============================================================
print("\n" + "=" * 60)
print("SECTION 3: PYQ Statistics and Search")
print("=" * 60)

run_test(
    name="PYQ statistics overview",
    method="GET",
    path="/api/pyq/statistics",
    expected_status=200
)

run_test(
    name="Search PYQ questions",
    method="GET",
    path="/api/pyq/search",
    expected_status=200,
    params={
        "exam_id": CONTEXT["competitive_exam_id"],
        "subject_id": CONTEXT["subject_id"],
        "page_size": 5
    }
)

# ============================================================
print("\n" + "=" * 60)
print("SECTION 4: PYQ Session Management")
print("=" * 60)

# Define session filters
pyq_session_filters = {
    "exam_id": CONTEXT["competitive_exam_id"],
    "subject_id": CONTEXT["subject_id"],
    "topic_id": CONTEXT["topic_id"],
    "shuffle_questions": False
}

# Create PYQ session
print("\n  → Creating PYQ practice session...")
pyq_session_resp = run_test(
    name="Create PYQ session",
    method="POST",
    path="/api/pyq/session/create",
    expected_status=201,
    json={
        "user_id": CONTEXT["test_user_id"],
        "session_name": f"Notebook PYQ Session {CONTEXT['unique_tag']}",
        "filters": pyq_session_filters,
        "time_limit": 30
    }
)

# Check if session was created successfully
session_created = isinstance(pyq_session_resp["body"], dict) and pyq_session_resp["body"].get("success")
if session_created:
    session = pyq_session_resp["body"].get("session", {})
    session_id = session.get("id")
    CONTEXT["pyq_session_id"] = session_id
    print(f"  ✅ Session created: {session_id}")
    print(f"     Total questions: {session.get('total_questions', 'N/A')}")
else:
    session_id = "00000000-0000-0000-0000-000000000000"
    CONTEXT["pyq_session_id"] = None
    print(f"  ❌ Session creation failed: {pyq_session_resp['body'].get('error', 'Unknown error')}")

# ============================================================
print("\n" + "=" * 60)
print("SECTION 5: Session Navigation and Question Access")
print("=" * 60)

# Get current question
current_note = None if session_created else "Session creation failed; placeholder session id used."
current_question_resp = run_test(
    name="Get current PYQ question",
    method="GET",
    path=f"/api/pyq/session/{session_id}/current",
    expected_status=200 if session_created else 400,
    extra_expected=None if session_created else 404,
    note=current_note
)

if session_created and isinstance(current_question_resp["body"], dict) and current_question_resp["body"].get("success"):
    current_question = current_question_resp["body"].get("question", {})
    CONTEXT["pyq_current_question_id"] = current_question.get("id")
    print(f"  → Current question ID: {CONTEXT['pyq_current_question_id']}")
else:
    CONTEXT["pyq_current_question_id"] = None

# Submit answer
question_id_for_submit = CONTEXT.get("pyq_current_question_id") or "00000000-0000-0000-0000-000000000000"
submit_expected = 200 if CONTEXT.get("pyq_current_question_id") else 400
submit_extra = None if submit_expected == 200 else 404
submit_note = None if submit_expected == 200 else "Submitting with placeholder identifiers to exercise the endpoint."

run_test(
    name="Submit PYQ answer",
    method="POST",
    path=f"/api/pyq/session/{session_id}/submit",
    expected_status=submit_expected,
    extra_expected=submit_extra,
    note=submit_note,
    json={
        "question_id": question_id_for_submit,
        "user_answer": "Option A",
        "time_taken": 45
    }
)

# Navigate to next question
navigate_note = None if session_created else "Placeholder session id used for navigation call."
run_test(
    name="Navigate PYQ session (next)",
    method="POST",
    path=f"/api/pyq/session/{session_id}/navigate/next",
    expected_status=200 if session_created else 400,
    extra_expected=None if session_created else 404,
    note=navigate_note
)

# Jump to specific question
run_test(
    name="Jump to PYQ question index",
    method="POST",
    path=f"/api/pyq/session/{session_id}/jump/0",
    expected_status=200 if session_created else 400,
    extra_expected=None if session_created else 404,
    note=navigate_note
)

# ============================================================
print("\n" + "=" * 60)
print("SECTION 6: Session Progress and State Management")
print("=" * 60)

# Get session progress
run_test(
    name="Get PYQ session progress",
    method="GET",
    path=f"/api/pyq/session/{session_id}/progress",
    expected_status=200 if session_created else 400,
    extra_expected=None if session_created else 404,
    note=navigate_note
)

# Pause session
print("\n  → Testing pause/resume functionality...")
pause_resp = run_test(
    name="Pause PYQ session",
    method="POST",
    path=f"/api/pyq/session/{session_id}/pause",
    expected_status=200 if session_created else 400,
    extra_expected=None if session_created else 404,
    note=navigate_note
)

# Resume session
resume_resp = run_test(
    name="Resume PYQ session",
    method="POST",
    path=f"/api/pyq/session/{session_id}/resume",
    expected_status=200 if session_created else 400,
    extra_expected=None if session_created else 404,
    note=navigate_note
)

# List user sessions
print("\n  → Listing all sessions for user...")
run_test(
    name="List PYQ sessions for user",
    method="GET",
    path=f"/api/pyq/sessions/user/{CONTEXT['test_user_id']}",
    expected_status=200
)

print("\n" + "=" * 60)
print("PYQ Endpoints Testing Complete")
print("=" * 60)

SECTION 1: PYQ Template and Filter Options
✅ Download PYQ template -> 200
✅ PYQ filter options -> 200

SECTION 2: Upload PYQ Questions
✅ Upload single PYQ question -> 201
  → Uploaded PYQ question ID: d94440c7-466c-442d-a036-d12bdba119e3
✅ Upload bulk PYQ questions -> 200
✅ Upload PYQ questions via Excel -> 200

SECTION 3: PYQ Statistics and Search
✅ PYQ statistics overview -> 200
✅ Search PYQ questions -> 200

SECTION 4: PYQ Session Management

  → Creating PYQ practice session...
✅ Create PYQ session -> 201
  ✅ Session created: ccc74eee-1690-414b-bacf-74eb364dad8b
     Total questions: 6

SECTION 5: Session Navigation and Question Access
✅ Get current PYQ question -> 200
  → Current question ID: 6d1b1ea0-6e2e-493f-bde9-ec766de1a62f
✅ Submit PYQ answer -> 200
✅ Navigate PYQ session (next) -> 200
✅ Jump to PYQ question index -> 200

SECTION 6: Session Progress and State Management
✅ Get PYQ session progress -> 200

  → Testing pause/resume functionality...
✅ Pause PYQ session -> 200
✅ 

In [58]:
# Summarise test results
results_df = pd.DataFrame(RESULTS)
display(results_df)

if not results_df.empty:
    print("Summary by expectation match:")
    display(results_df.groupby("ok").size().rename("count"))
    failing = results_df[~results_df["ok"]]
    if not failing.empty:
        print("Endpoints with unexpected status codes:")
        display(failing[["name", "method", "path", "status", "expected", "note"]])



,name,method,path,status,expected,ok,note,request_payload,response_preview
0,Create school exam,POST,/api/exams,201,[201],True,None,{'name': 'Notebook School Exam 20251004080817'...,"[{""created_at"": ""2025-10-04T08:08:17.62+00:00""..."
1,Create competitive exam,POST,/api/exams,201,[201],True,None,{'name': 'Notebook Competitive Exam 2025100408...,"[{""created_at"": ""2025-10-04T08:08:17.771544+00..."
2,Create school class,POST,/api/classes,201,[201],True,None,{'exam_id': '0c8b6307-fd0a-42f1-ab94-3de94d06c...,"[{""class_number"": 10, ""created_at"": ""2025-10-0..."
3,Create competitive subject,POST,/api/subjects,201,[201],True,None,"{'name': 'Notebook Subject cada2316', 'descrip...","[{""class_id"": null, ""created_at"": ""2025-10-04T..."
4,Create school subject,POST,/api/subjects,201,[201],True,None,"{'name': 'Notebook School Subject cada2316', '...","[{""class_id"": ""33d3f1ff-a0cb-4fee-9d29-11ed4d9..."
5,Create chapter,POST,/api/chapters,201,[201],True,None,{'subject_id': 'b03320be-fbe3-4f8a-9feb-147c2e...,"[{""created_at"": ""2025-10-04T08:08:18.363327+00..."
6,Create topic,POST,/api/topics,201,[201],True,None,{'chapter_id': '42418a13-2000-4879-aa52-a20230...,"[{""chapter_id"": ""42418a13-2000-4879-aa52-a2023..."
7,Create concept,POST,/api/concepts,201,[201],True,None,{'topic_id': '1a499dcc-6d5e-438f-9130-4a6a7c70...,"[{""created_at"": ""2025-10-04T08:08:18.66763+00:..."
8,Add topic attributes,POST,/api/topic/1a499dcc-6d5e-438f-9130-4a6a7c700e8...,201,[201],True,None,"[{'name': 'Notebook Attribute cada2316', 'desc...","{""attributes"": [{""created_at"": ""2025-10-04T08:..."
9,List topic attributes,GET,/api/topic/1a499dcc-6d5e-438f-9130-4a6a7c700e8...,200,[200],True,None,None,"[{""description"": ""Primary attribute for notebo..."


Summary by expectation match:


ok
True    48
Name: count, dtype: int64

In [72]:
import requests

# Upload question image
files = {'image': open('ailt.jpg', 'rb')}
response = requests.post(
    f'http://localhost:5200/api/questions/d6c0cdad-91e9-4e68-8d0a-2f089a98e8ae/image',
    files=files
)

In [75]:
files = {
    'option_A': open('ailt.jpg', 'rb'),
    'option_B': open('ailt.jpg', 'rb')
}
requests.post(f'http://localhost:5200/api/questions/d6c0cdad-91e9-4e68-8d0a-2f089a98e8ae/options/images', files=files)


<Response [200]>

In [73]:
response.text

'{"message":"Image uploaded successfully","path":"questions/d6c0cdad-91e9-4e68-8d0a-2f089a98e8ae/question.jpg","success":true,"url":"https://enilfsnxhqcafhigmzsc.supabase.co/storage/v1/object/public/question-images/questions/d6c0cdad-91e9-4e68-8d0a-2f089a98e8ae/question.jpg"}\n'

In [64]:
files

{'image': <_io.BufferedReader name='ailt.jpg'>}

In [61]:
results_df.iloc[19,-1]

'{"attributes": [{"description": "Primary attribute for notebook questions", "id": "d6c0cdad-91e9-4e68-8d0a-2f089a98e8ae", "name": "Notebook Attribute cada2316", "value": true}], "chapter_id": "42418a13-2000-4879-aa52-a20230bd7ab8", "class_id": null, "concept_id": "d8117670-972f-476f-8b83-62e869710fc3", "content": "What is the notebook marker cada2316?", "correct_answer": "Option A", "created_at": ...'